In [1]:
import os
#conda_lib_path = "/home/tristan/.conda/envs/jphydro/lib/libstdc++.so.6"
#os.environ["LD_PRELOAD"] = conda_lib_path

#from imports import *
from hydro_data.config import DATASET_VARIABLES
from hydro_data.interpolation.aggregation import aggregate_all_grids, aggregate_all_fcst
from hydro_data.interpolation.postprocessing import postprocess_yearly_all_grid
from hydro_data.interpolation.weight_precomp import precompute_all_weights, TS_ROOTS
from hydro_data.data_io import load_graph_data, GRAPH_ROOT
from shapely.geometry import MultiPolygon

import numpy as np

import xarray as xr
from pathlib import Path
import rasterio
from tqdm.auto import tqdm

import geopandas as gpd
from shapely.geometry import box
import hvplot.pandas

from hydro_data.interpolation.weight_precomp import _compute_polygon_weights_wrapper
from hydro_data.interpolation.weight_precomp import *

def filter_polygone(poly):
    if isinstance(poly, MultiPolygon):
        return poly.geoms[np.argmax([x.area for x in poly.geoms])]
    else:
        return poly

def load_geometries(graph_name):
    g, points, catchments, kp = load_graph_data(graph_name)
    idx = catchments[catchments.area==0].index
    catchments.loc[idx, "geometry"] = points.loc[idx].buffer(.0001)
    return catchments["geometry"].apply(filter_polygone)

def load_ds(fp):
    ds = xr.open_dataset(fp).transpose("y", "x", "time")
    ds["x"]=ds["x"]*100
    ds["y"]=ds["y"]*100
    return ds

In [2]:
data_root = Path("../data")
n_jobs = 85

In [3]:
for fp in tqdm((data_root / "SIM_processed").glob("*.nc")):
    ds = load_ds(fp)
    break

0it [00:02, ?it/s]


In [4]:
gdf = load_geometries("FR_10_base").to_crs("EPSG:27572")

/tmp/ipykernel_2218288/2640702512.py:35: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  idx = catchments[catchments.area==0].index


In [5]:
X = ds["T"].values 
msk = np.where(~(np.isnan(X).any(-1)))
msk

(array([  0,   1,   1, ..., 132, 132, 133], shape=(9892,)),
 array([ 70,  66,  67, ..., 139, 140, 139], shape=(9892,)))

In [42]:
#msk = np.where(np.ones_like(X).sum(-1))

In [6]:
# da: xarray DataArray with coords 'x' and 'y'
xmin = float(ds["x"].min())
xmax = float(ds["x"].max())
ymin = float(ds["y"].min())
ymax = float(ds["y"].max())

grid_poly = box(xmin, ymin, xmax, ymax)
#gdf = gdf[gdf.within(grid_poly)]

In [7]:
polygon = gdf.values[0]

In [8]:
grid_lat, grid_lon = ds.y.values, ds.x.values

In [9]:
#compute_polygon_weights(polygon, grid_lat, grid_lon, msk)

In [10]:
weight = compute_weights(gdf, ds.y.values, ds.x.values, msk, n_jobs=n_jobs)

Computing weights: 100%|██████████| 35083/35083 [03:07<00:00, 187.49it/s]


In [11]:
#compute_weights??
weight["polygon_index"].unique()

array([     8275,      8782,    214845, ...,      8727,  75493187,
       159120636], shape=(35083,))

### Interpolate

In [12]:
def aggregate_da(da, weights):
    x = da.values[weights.i, weights.j].T
    x = x * weights.norm_weight.values
    x = pd.DataFrame(x.T, index=weights.index, columns=[da.time.to_pandas()])
    x = x.groupby(weights["polygon_index"]).sum().T
    x.index = x.index.get_level_values(0)
    return x

In [13]:
var = ["PRENEI", "PRELIQ", "PE", "T", "EVAP", "RUNC", "RESR_NEIGE"]

In [14]:
dss = []
for fp in tqdm((data_root / "SIM_processed").glob("*.nc")):
    ds = load_ds(fp)
    #x.index.name = "time"
    new_ds = xr.Dataset({v: aggregate_da(ds[v], weight) for v in tqdm(var)})
    dss.append(new_ds)

0it [00:00, ?it/s]
100%|██████████| 7/7 [00:30<00:00,  4.30s/it]
1it [00:30, 30.12s/it]
100%|██████████| 7/7 [00:29<00:00,  4.21s/it]
2it [00:59, 29.90s/it]
100%|██████████| 7/7 [00:19<00:00,  2.74s/it]
3it [01:19, 25.14s/it]
100%|██████████| 7/7 [00:28<00:00,  4.11s/it]
4it [01:48, 26.77s/it]
100%|██████████| 7/7 [00:28<00:00,  4.14s/it]
5it [02:17, 27.67s/it]
100%|██████████| 7/7 [00:27<00:00,  3.96s/it]
6it [02:45, 27.65s/it]


In [ ]:
ds = xr.concat(dss, dim="time").sortby("time")

In [ ]:
ds.to_netcdf(data_root / "dataset" / "inp_dyn_.nc")

In [19]:
data_root / "dataset" / "inp_dyn_.nc"

PosixPath('../data/dataset/inp_dyn_.nc')

In [ ]:
x = aggregate_da(ds[v], weight)

In [ ]:
#graphs = ["JFlow_10_latest", "JFlow_5_latest"] 
#graphs = ["JFlow_40_v1", "JFlow_20_v1", "JFlow_10_v1", "JFlow_5_v1"]  # 

DATASET_VARIABLES = { 
    #"oki"     : ["prcp"], 
    #"era"     : ['u10', 'v10', 'd2m', 't2m', 'pev', 'sd', 'ssr', 'str', 'sp', 'tp', 'swvl1', 'swvl2', 'swvl3', 'swvl4'],
    #"msm_a"   : ['psea', 'sp', 'u', 'v', 'rh', 'ncld_upper', 'ncld_mid', 'ncld_low', 'ncld'],
    #"garadar" : ["prcp"],
    "radar"   : ["prcp"], 
    #"msm_a"   : ["temp", "r1h"],
}
years =list(range(2008, 2021))#   None#[2015, 2017, 2023] #list(range(2008, 2025))

for data_name, variables in DATASET_VARIABLES.items():
    print(f"Interpolate: {data_name} ...")
    aggregate_all_grids(data_name, variables, graphs, years=years)

Interpolate: radar ...


  0%|▎                                                                                                                                                                                                                                                        | 71/52695 [00:22<3:04:33,  4.75it/s]Ignoring index file '/data_prediction005/SYSTEM/prediction002/home/tristan/data/JMA/radar/original/radar/Z__C_RJTD_20080101115000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin.5b7b6.idx' incompatible with GRIB file


In [ ]:
weights, fps = init_aggregation(data_name, graphs, variables)

In [8]:
postprocess_yearly_all_grid(graphs, DATASET_VARIABLES)

Postprocessing ('JFlow_10_v1', 'radar', 'prcp')


100%|███████████████████████████████████████████| 13/13 [02:26<00:00, 11.29s/it]


Dumping ('JFlow_10_v1', 'radar', 'prcp', '1h') ...


100%|████████████████████████████████████| 24048/24048 [02:17<00:00, 174.67it/s]


Dumping ('JFlow_10_v1', 'radar', 'prcp', '6h') ...


...

100%|████████████████████████████████████| 16080/16080 [01:07<00:00, 237.78it/s]


Dumping ('JFlow_20_v1', 'radar', 'prcp', '6h') ...


100%|████████████████████████████████████| 16080/16080 [00:16<00:00, 995.82it/s]


Dumping ('JFlow_20_v1', 'radar', 'prcp', '1d') ...


100%|███████████████████████████████████| 16080/16080 [00:07<00:00, 2111.87it/s]


### Dataset Assembly